# US Upper (LangChain)

In [172]:
from pyprojroot.here import here
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_core.output_parsers import StrOutputParser
from _utils import read_markdown

# base_path = pyprojroot.find_root(pyprojroot.has_dir(".git"))

## Prompt

### Construct Prompt Template 

In [211]:
pr_text_main = read_markdown(here("prompt-lc/main.md"))
# pr_text_eng_style_guide = read_markdown(here("prompt-lc/eng_style_guide.md"))
# pr_text_report_structure = read_markdown(here("prompt-lc/report_structure.md"))
# pr_text_report_template_normal = read_markdown(here("prompt-lc/report_template_normal.md"))
# print(prompt_main_str)

In [209]:
pr_temp = ChatPromptTemplate.from_template(pr_text_main)
pr_temp

ChatPromptTemplate(input_variables=['abnormal_gallbladder', 'abnormal_kidney', 'abnormal_liver', 'user'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['abnormal_gallbladder', 'abnormal_kidney', 'abnormal_liver', 'user'], template='You are a radiology report writer in my institution. \n\nI will provide you:\n\n- "English Style Guide" for the preferred ways to write phrases or sentences in the report.\n- "Reporting Structure" provides blueprint to build radiology report for each studies.\n- "Normal Report Template" provides normal reporting template and normal findings for each studies.\n- "Abnormal Report Template" provide template to write abnormal findings and corresponding impression for each specific organs and conditions.\n\n\nUser role: \n- The user (radiologist) will provide you with ultrasound findings.\n- If findings for each specific organ is not provide, assume normal findings for that organ. \n- If the user ask "How do I use you?", provide the 

In [208]:
# Not Work when execute chain
# pr_temp2 = ChatPromptTemplate.from_messages([
#     ("system", pr_text_main),
#     MessagesPlaceholder("msgs")
# ])
# pr_temp2

ChatPromptTemplate(input_variables=['abnormal_gallbladder', 'abnormal_kidney', 'abnormal_liver', 'msgs', 'user'], input_types={'msgs': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['abnormal_gallbladder', 'abnormal_kidney', 'abnormal_liver', 'user'], template='You are a radiology report writer in my institution. \n\nI will provide you:\n\n- "English Style Guide" for the preferred ways to write phrases or sentences in the report.\n- "Reporting Structure" provides blueprint to build radiology report for each studies.\n- "Normal Report Template" provides normal reporting template and normal findings for each studies.\n- "Abnormal Report Template" provide template to write a

### Test invoke prompt value

In [210]:
pr_val_1 = pr_temp.invoke({
               "abnormal_gallbladder": "...",
               "abnormal_kidney": "...",
               "abnormal_liver": "...",
               "user": [HumanMessage(content="hi!")]})

pr_val_1.to_messages()
# print(pr_val_1.to_string())

[HumanMessage(content='You are a radiology report writer in my institution. \n\nI will provide you:\n\n- "English Style Guide" for the preferred ways to write phrases or sentences in the report.\n- "Reporting Structure" provides blueprint to build radiology report for each studies.\n- "Normal Report Template" provides normal reporting template and normal findings for each studies.\n- "Abnormal Report Template" provide template to write abnormal findings and corresponding impression for each specific organs and conditions.\n\n\nUser role: \n- The user (radiologist) will provide you with ultrasound findings.\n- If findings for each specific organ is not provide, assume normal findings for that organ. \n- If the user ask "How do I use you?", provide the "User guide", or if not provided, generate it.\n\nYour task: \n- Build radiology report using "Reporting Structure", "Normal Report Template", and "Abnormal Findings Report Template" with language style according to the "english style guid

## Documents 

### Load Markdown Docs (Single)
May not be a good ideas, since each organs must be filled 

In [83]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("docs/report_template_abnormal.md")
doc_abnormal = loader.load()

In [84]:
doc_abnormal[0]

Document(page_content='# Abnormal Findings\n\n\n## Liver Findings\n\n### Parenchymatous Liver Disease\n\n```markdown\n**Liver:** Normal size and (mildly) `[increased | coarse]` parenchymal echogenicity. No focal lesion.\n**IMPRESSION:**\n- (Mild) parenchymatous disease of the liver without focal lesion.\n```\n\n### Fatty Liver\n\n#### Mild Fatty Liver\n\n```markdown\n**Liver:** Normal size with mildly increased parenchymal echogenicity of the liver. No focal lesion.\n**IMPRESSION:**\n- Mild fatty liver without focal lesion.\n```\n\n#### Moderate Fatty Liver\n\n```markdown\n**Liver:** Normal size with diffusely increased parenchymal echogenicity of the liver, causing imparied visualization of intrahepatic vasculature. No focal lesion.\n**IMPRESSION:**\n- Moderate fatty liver without focal lesion.\n```\n\n#### Severe Fatty Liver\n\n```markdown\n**Liver:** Normal size with diffusely increased parenchymal echogenicity of the liver, causing imparied visualization of intrahepatic vasculature

### Load Markdown Docs (Multiple)

In [114]:
## From Folders
loader = DirectoryLoader("docs/by_organs", glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()
len(docs)
# docs[0].page_content
# docs[0].metadata
docs

[Document(page_content='# Kidney Findings\n\n\nOrder findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.\n\n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```\n\n\n### (Chronic) Parenchymatous Kidney Disease\n\nDefinition: \n\n"Parenchymatous kidney disease" := normal kidney size but increased echogenicity. \n"Chronic parenchymatous kidney disease" := small kidney size and increased echogenicity. \n\nIf one kidney is abnormal and the other is normal, report findings for each kidneys. \n\nHere is the format:\n\n```markdown\n**Kidneys:** `[Normal | Small]` size with (mildly) increased parenchymal echogenicity of the `[right | left | both]` kidney(s). No stone, hydronephrosis or solid mass.\n**IMPRESSION:**\n- (Chronic) parenchymatous disease of `[right | left | both]` kidney(s).\n```\n\nExamples:\n\n- Parenchymatous right kidney and normal left kidney:\n\n```markdow

In [124]:
from pathlib import Path
docs_names = [Path(doc.metadata["source"]).stem for doc in docs]

docs_dict = dict(zip(docs_names, docs))
docs_dict

{'abnormal_kidney': Document(page_content='# Kidney Findings\n\n\nOrder findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.\n\n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```\n\n\n### (Chronic) Parenchymatous Kidney Disease\n\nDefinition: \n\n"Parenchymatous kidney disease" := normal kidney size but increased echogenicity. \n"Chronic parenchymatous kidney disease" := small kidney size and increased echogenicity. \n\nIf one kidney is abnormal and the other is normal, report findings for each kidneys. \n\nHere is the format:\n\n```markdown\n**Kidneys:** `[Normal | Small]` size with (mildly) increased parenchymal echogenicity of the `[right | left | both]` kidney(s). No stone, hydronephrosis or solid mass.\n**IMPRESSION:**\n- (Chronic) parenchymatous disease of `[right | left | both]` kidney(s).\n```\n\nExamples:\n\n- Parenchymatous right kidney and normal left ki

### Split Markdown by Headers

In [248]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=True
)
md_header_splits_dict = {name: markdown_splitter.split_text(doc.page_content) for name, doc in docs_dict.items()}
md_header_splits_dict

{'abnormal_kidney': [Document(page_content='Order findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.  \n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```', metadata={'Header 1': 'Kidney Findings'}),
  Document(page_content='Definition:  \n"Parenchymatous kidney disease" := normal kidney size but increased echogenicity.\n"Chronic parenchymatous kidney disease" := small kidney size and increased echogenicity.  \nIf one kidney is abnormal and the other is normal, report findings for each kidneys.  \nHere is the format:  \n```markdown\n**Kidneys:** `[Normal | Small]` size with (mildly) increased parenchymal echogenicity of the `[right | left | both]` kidney(s). No stone, hydronephrosis or solid mass.\n**IMPRESSION:**\n- (Chronic) parenchymatous disease of `[right | left | both]` kidney(s).\n```  \nExamples:  \n- Parenchymatous right kidney and normal left kidney:  \

### Vector Store

In [249]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

chroma_dict = {name: Chroma.from_documents(md_header_splits, embedding=OpenAIEmbeddings()) 
               for name, md_header_splits in md_header_splits_dict.items()}

## Retriver

In [250]:
retriever_dict = {
    name: chroma.as_retriever(
    search_type="similarity",
    # For Maximum diversity
    search_kwargs={'k': 6},) 
    for name, chroma in chroma_dict.items()
                  }

retriever_dict["abnormal_gallbladder"]

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x16ad440a0>, search_kwargs={'k': 6})

In [243]:
retriever_dict["abnormal_gallbladder"].invoke("What is template for gallstone?")

[Document(page_content='### Gallstone(s)  \n```markdown\n**Gallbladder:** Distended gallbladder containing `[a ?-cm | a few | many ]` gallstone(s), (measuring up to ___ cm). No gallbladder wall thickening or pericholecystic fluid. No mass\n**IMPRESSION:**\n- `[a ?-cm | a few | many ]` gallstone(s) without evidence of cholecystitis\n```', metadata={'Header 1': 'Gallbladder Abnormal Findings', 'Header 3': 'Gallstone(s)'}),
 Document(page_content='```markdown\n**Gallbladder:** Distended gallbladder containing `[a ?-cm | a few | many ]` gallstone(s), (measuring up to ___ cm). No gallbladder wall thickening or pericholecystic fluid. No mass\n**IMPRESSION:**\n- `[a ?-cm | a few | many ]` gallstone(s) without evidence of cholecystitis\n```', metadata={'Header 1': 'Abnormal Findings', 'Header 2': 'Gallbladder Findings', 'Header 3': 'Gallstone(s)'}),
 Document(page_content='```markdown\n**Gallbladder:** Distended gallbladder containing `[a ?-cm | a few | many ]` gallstone(s), (measuring up to _

In [244]:
retriever_dict["abnormal_kidney"].invoke("Find renal stones template")

[Document(page_content='Order findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.  \n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```', metadata={'Header 1': 'Abnormal Findings', 'Header 2': 'Kidney Findings'}),
 Document(page_content='Order findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.  \n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```', metadata={'Header 1': 'Kidney Findings'}),
 Document(page_content='### Renal Stone  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s).  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s)\n```  \

## Chain 

In [231]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [252]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"abnormal_gallbladder": retriever_dict["abnormal_gallbladder"] | format_docs,
     "abnormal_kidney": retriever_dict["abnormal_kidney"] | format_docs,
     "abnormal_liver": retriever_dict["abnormal_liver"] | format_docs,
     "user": RunnablePassthrough()
     }
    | pr_temp
    | llm
    | StrOutputParser()
)

## Execute !

In [253]:
res0 = rag_chain.invoke("Generate normal US report")
print(res0)

**US OF THE UPPER ABDOMEN**

**FINDINGS:**

**Liver:** Normal size and parenchymal echogenicity. No focal lesion.
**Biliary system:** CBD size ___ mm. No intrahepatic ductal dilatation.
**Gallbladder:** Well-distended gallbladder. No stone or mass.
**Spleen:** Normal in size.
**Pancreas:** Visualized portions are unremarkable.
**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. No stone, hydronephrosis or solid mass.
**Aorta:** Normal caliber.

**IMPRESSION:**
- Normal liver parenchyma without focal lesion.


In [254]:
pr_user_upper1 = """Generate US report with these findings:
- Severe fatty liver
"""
res1 = rag_chain.invoke(pr_user_upper1)
print(res1)

**US OF THE UPPER ABDOMEN**

**FINDINGS:**

**Liver:** Normal size with diffusely increased parenchymal echogenicity of the liver, causing impaired visualization of intrahepatic vasculature and right hemidiaphragm. No focal lesion.
**Biliary system:** CBD size ___ mm. No intrahepatic ductal dilatation.
**Gallbladder:** Well-distended gallbladder. No stone or mass.
**Spleen:** Normal in size.
**Pancreas:** Visualized portions are unremarkable.
**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. No stone, hydronephrosis or solid mass.
**Aorta:** Normal caliber.

**IMPRESSION:**
- Severe fatty liver without focal lesion.


In [255]:
pr_user_upper2 = """Generate US report with these findings:
- Severe fatty liver, CBD 3 mm
- Chronic parenchymatous both kidneys
- A 3-cm simple left renal cyst 
"""
# Not all
res2 = rag_chain.invoke(pr_user_upper2)
print(res2)

**US OF THE UPPER ABDOMEN**

**FINDINGS:**

**Liver:** Normal size with diffusely increased parenchymal echogenicity of the liver, causing impaired visualization of intrahepatic vasculature and right hemidiaphragm. No focal lesion.
**Biliary system:** Dilated CBD, measures about 3 mm without demonstrable cause of obstruction. No intrahepatic ductal dilatation.
**Gallbladder:** Well-distended gallbladder. No stone or mass.
**Spleen:** Normal in size.
**Pancreas:** Visualized portions are unremarkable.
**Kidneys:** Small size with increased parenchymal echogenicity of both kidneys. A 3-cm simple left renal cyst. No stone, hydronephrosis or solid mass.
**Aorta:** Normal caliber.

**IMPRESSION:**
- Severe fatty liver with impaired visualization of intrahepatic vasculature and right hemidiaphragm.
- Chronic parenchymatous disease of both kidneys.
